In [1]:
import os
import numpy as np 
import pandas as pd
import shutil as sh
from PIL import Image
from tqdm.auto import tqdm

In [8]:
df = pd.read_csv('./train/via_export_csv.csv')
df

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,image-person-0001.png,2422705,{},3,0,"{""name"":""rect"",""x"":387,""y"":290,""width"":189,""he...","{""name"":""person""}"
1,image-person-0001.png,2422705,{},3,1,"{""name"":""rect"",""x"":861,""y"":287,""width"":233,""he...","{""name"":""person""}"
2,image-person-0001.png,2422705,{},3,2,"{""name"":""rect"",""x"":1175,""y"":274,""width"":187,""h...","{""name"":""person""}"
3,image-person-0002.png,2431238,{},6,0,"{""name"":""rect"",""x"":11,""y"":157,""width"":289,""hei...","{""name"":""person""}"
4,image-person-0002.png,2431238,{},6,1,"{""name"":""rect"",""x"":308,""y"":85,""width"":355,""hei...","{""name"":""person""}"
5,image-person-0002.png,2431238,{},6,2,"{""name"":""rect"",""x"":677,""y"":143,""width"":251,""he...","{""name"":""person""}"
6,image-person-0002.png,2431238,{},6,3,"{""name"":""rect"",""x"":1125,""y"":155,""width"":44,""he...","{""name"":""person""}"
7,image-person-0002.png,2431238,{},6,4,"{""name"":""rect"",""x"":913,""y"":129,""width"":371,""he...","{""name"":""person""}"
8,image-person-0002.png,2431238,{},6,5,"{""name"":""rect"",""x"":1409,""y"":135,""width"":479,""h...","{""name"":""person""}"
9,image-person-0003.png,2303856,{},6,0,"{""name"":""rect"",""x"":62,""y"":129,""width"":452,""hei...","{""name"":""person""}"


In [9]:
## create x, y, w, h columns 
x, y, w, h = [], [], [], []
for row in df['region_shape_attributes']:
    row = row.replace('{}', '').replace('}', '')
    row = row.split(',')
    x.append(int(row[1].split(':')[-1]))
    y.append(int(row[2].split(':')[-1]))
    w.append(int(row[3].split(':')[-1]))
    h.append(int(row[4].split(':')[-1]))
    
## calculating x, y, width and height coordinates
df['x'], df['y'], df['w'], df['h'] = x, y, w, h

## creating a column name image_id having images names as id 
df['image_id'] = [name.split('.')[0] for name in df['filename']]

## creating two columns for storing x and y center values
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2

## define number of classes 
labels = df['region_attributes'].unique()
labels_to_dict = dict(zip(labels, range(0, len(labels))))
print('Labels Directory:', labels_to_dict)

df['classes'] = df['region_attributes']
df.replace({'classes':labels_to_dict}, inplace=True)
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

## set index of images
index = list(set(df.image_id))

Labels Directory: {'{"name":"person"}': 0, '{"name":"gazelle"}': 1, '{"name":"leopard"}': 2}


In [11]:
#copy images from one dir to another
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val/'
            else:
                path2save = 'train/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                imagename = "./train/images/{}.png".format(name)
                check_image_width_height = Image.open(imagename)
                img_width, img_height = check_image_width_height.size
                for r in (row):
                    r[1] = r[1]/img_width
                    r[2] = r[2]/img_height
                    r[3] = r[3]/img_width
                    r[4] = r[4]/img_height
                row = row.astype(str)
                for j in range(len(row)):
                    print(row[j], '\n')
                    row[j][0] = str(int(float(row[j][0])))
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("./train/images/{}.png".format(name),'convertor/fold{}/images/{}/{}.png'.format(fold,path2save,name))

  0%|          | 0/14 [00:00<?, ?it/s]

['1.0' '0.44244791666666666' '0.5953703703703703' '0.6505208333333333'
 '0.6777777777777778'] 

['1.0' '0.7880208333333333' '0.5101851851851852' '0.38333333333333336'
 '0.42407407407407405'] 

['1.0' '0.2109375' '0.5305555555555556' '0.25833333333333336'
 '0.40555555555555556'] 

['1.0' '0.425' '0.5527777777777778' '0.28229166666666666'
 '0.39444444444444443'] 

['1.0' '0.51796875' '0.21064814814814814' '0.13385416666666666'
 '0.2861111111111111'] 

['1.0' '0.38255208333333335' '0.2412037037037037' '0.13072916666666667'
 '0.2175925925925926'] 

['1.0' '0.8950520833333333' '0.09027777777777778' '0.10260416666666666'
 '0.14537037037037037'] 

['1.0' '0.44921875' '0.07407407407407407' '0.0828125'
 '0.11851851851851852'] 

['1.0' '0.6169270833333333' '0.10740740740740741' '0.0765625'
 '0.13333333333333333'] 

['1.0' '0.5377604166666666' '0.08194444444444444' '0.06927083333333334'
 '0.13796296296296295'] 

['1.0' '0.5130208333333334' '0.5365740740740741' '0.15104166666666666'
 '0.4064814814